In [1]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np
import scipy.stats as stats


# Testing
<details open>
  <summary><b>Simple Linear Regression</b></summary>
  &emsp;To predict the value of a continuous dependent variable based on one continuous independent variable.</br>
  &emsp;It models the relationship between two variables by fitting a straight line.
</details>

<details open>
  <summary><b>Multiple Linear Regression</b></summary>
  &emsp;To predict a continuous dependent variable based on two or more independent variables (which can be continuous or categorical).</br>
  &emsp;It extends simple regression to multiple predictors to understand their combined effect.
</details>

<details open>
  <summary><b>Contingency Table (Chi-square Test)</b></summary>
  &emsp;To examine the relationship between two categorical variables.</br>
  &emsp;It tests whether distributions of categorical variables differ from each other.
</details>

<details open>
  <summary><b>t-test (One Sample)</b></summary>
  &emsp;To determine whether the mean of a single sample is significantly different from a known or hypothesized population mean.</br>
  &emsp;It compares the sample mean to a specific value.
</details>

<details open>
  <summary><b>Welch t-test (Two Sample)</b></summary>
  &emsp;To compare the means of two independent groups to see if they are statistically different when variances are unequal.</br>
  &emsp;It evaluates whether the difference between two groups’ means is statistically significant.
   It adjusts for differences in variance and sample size between the two groups.
</details>

<details open>
  <summary><b>Pooled t-test (Two Sample)</b></summary>
  &emsp;To compare the means of two independent groups to see if they are statistically different when variances are assumed to be equal.</br>
  &emsp;It evaluates whether the difference between two groups’ means is statistically significant.
   It combines (pools) variances from both groups for a more precise estimate.
</details>

<details open>
  <summary><b>Paired t-test</b></summary>
  &emsp;To compare means from the same group at two different times (repeated measures) or under two different conditions.</br>
  &emsp;It analyzes differences within paired observations.
</details>

<details open>
  <summary><b>ANOVA (One-way Analysis of Variance)</b></summary>
  &emsp;To compare the means of three or more independent groups based on one independent variable.</br>
  &emsp;It tests if at least one group mean is different from the others.
</details>

<details open>
  <summary><b>ANOVA (Two-way Analysis of Variance)</b></summary>
  &emsp;To examine the effect of two independent variables on one continuous dependent variable and their interaction effect.</br>
  &emsp;It checks for main effects and interactions between two factors.
</details>

<details open>
  <summary><b>Chi-Square test</b></summary>
  &emsp;To check if two categorical variables are independent.</br>
  &emsp;It checks for main effects and interactions between two factors.
</details>


#### Error types
**Type I:** Falsely rejecting $H_0$ (False positive)</br>
**Type II:** Falsely rejecting $H_1$ (False negative)

## Simple Linear Regression

This is creation of a model with a linear trend. Thus always taking the form
$$y=f(x)=\beta_0+\beta_1x$$
When the model is based of a sample, there will not fit perfectly and we will need an error term:
$$Y_i=\beta_0+\beta_1x_i+\epsilon_i,~~i=\{1,\dots,n\}$$
We assume these errors ($\epsilon_i$) are normally distributed and independent.

Creation of this model is a process of evaluating $\beta_0$ and $\beta_1$. This process is generally estimated using a sample and thus we only achieve approximations:
$$\hat y_i=\hat\beta_0+\hat\beta_1x_i$$
For a successful approximation, it is expected that:
$$E(Y_i)=\beta_0+\beta_1x_i$$

In [ ]:
# Manual Calculation
x = np.array([168, 161, 167, 179, 184, 166, 198, 187, 191, 179]) # first values e.g. height
y = np.array([65.5, 58.3, 68.1, 85.7, 80.5, 63.4, 102.6, 91.4, 86.7,78.9]) # e.g. weight
# Calculate averages
xbar = x.mean()
ybar = y.mean()
# Parameters estimates
Sxx = sum((x - xbar)**2)
beta1hat = sum((x - xbar)*(y - ybar)) / Sxx
beta0hat = ybar - beta1hat * xbar
print('b0:', round(beta0hat.mean(),2), 'b1:', round(beta1hat.mean(),2))

In [ ]:
# Linear Regression with built in function
x = np.array(list(range(1990,2016))) # first values e.g. height
y = np.array([1.52, 2.88, 1.60, 2.24, 2.45, 1.84, 2.03, 2.33, 2.81, 2.46, 2.36, 2.23, 2.81, 2.70, 2.63, 2.00, 2.40, 2.45, 2.48, 2.51, 2.55, 2.77, 2.70, 2.23, 2.88, 3.09]) # e.g. weight

sample = pd.DataFrame({'x': x, 'y': y})
fitSample = smf.ols(formula = 'y ~ x', data=sample).fit()
print(fitSample.summary(slim=True))

In [ ]:
# Find new data point with model
new_x = pd.DataFrame({'x': [2022, 2023, 2024]})
pred = fitSample.get_prediction(new_x)
pred_frame = pred.summary_frame(alpha=0.05)
print(pred_frame[['obs_ci_lower', 'mean', 'obs_ci_upper']])


## Multiple Linear Regression

Linear Regression for multiple variables. It evaluates the combined effect of multiple variables on some value. The general model looks as such:
$$Y_i=\beta_0+\beta_1x_1+\dots+\beta_px_{p,i}+\epsilon_i,~~\epsilon_i\sim N(0,\sigma^2)$$
We assume the errors ($\epsilon_i$) are normally distributed and independent, with a mean of 0 and some unknown constant variance (identical for each contributing variable $x_n$).</br>
Degrees of freedom: $\text{df}=N-k-1$ where $N$ is observations and $k$ is number of variables


In [ ]:
# Parameter Estimation and Fitting
data = {
'y': [0.156, 1.234, 0.490, 1.649, 0.500, 0.395, 1.452, 0.416, 1.390,
0.234, 1.574, 0.349, 1.287, 1.709, 0.323, 1.201, 1.210, 1.787,
0.591, 0.110],
'x1' : [0.083, 0.409, 0.515, 0.397, 0.223, 0.292, 0.584, 0.491, 0.923,
0.280, 0.772, 0.857, 0.758, 0.850, 0.409, 0.055, 0.578, 0.745,
0.886, 0.031],
'x2' : [0.625, 0.604, 0.077, 0.414, 0.343, 0.202, 0.840, 0.266, 0.831,
0.385, 0.821, 0.308, 0.440, 0.865, 0.111, 0.970, 0.192, 0.939,
0.149, 0.318]}
df = pd.DataFrame(data)
# Parameter estimation
fit = smf.ols(formula = 'y ~ x1 + x2', data = df).fit()
# Summary of fit (parameter estimates, standard error, p-values, etc.)
print(fit.summary(slim=True))

In [ ]:
# Confidence Intervals for Parameters
print(fit.conf_int(alpha=0.05))

#### Backwards Selection (Simplification)
Once a model has been made, we can potentially simplify the model using backwards selection.</br>
The variable of least significance is chosen (highest p-value), removed and the model is re-evaluated. This is done until a significant difference is found.

## One Sample t-Test

Most commonly used in a simple hypothesis test, comparing a value (e.g. mean) to a specific constant:
$$H_0: \mu=\mu_0$$
The alternative hypothesis is that these are not equal.</br>
In such a test, the df is always one less than the sample size:
$$\text{df}=n-1$$
This test provides a *p-value*, which indicates the likelihood of our data assuming $H_0$ is true. If this value is too low, $H_0$ must be rejected. The threshold is decided by a chosen $\alpha$ value, of generally 0.01, or 0.05.

In [ ]:
μ0 = 1.5
alpha = 0.05
x = np.array([1.2, 2.4, 1.3, 1.3, 0.9, 1.0, 1.8, 0.8, 4.6, 1.4])

t_result = stats.ttest_1samp(x, popmean=μ0)

print("t: " + str(round(t_result.statistic, 4)))
print("p: " + str(round(t_result.pvalue, 4)))
ci = t_result.confidence_interval(1 - alpha)
print(f"{(1 - alpha) * 100:.0f}% CI for mean: ({round(ci[0], 4)}, {round(ci[1], 4)})")


In [ ]:
# test with summary statistics
x̄ = 34.66      # sample mean
s = 10.12          # sample standard deviation (unbiased)
n = 40            # sample size
μ0 = 30        # null hypothesis mean
alpha = 0.01      # significance level

se = s / np.sqrt(n)
t_stat = (x̄ - μ0) / se
df = n - 1
p_value = 2 * stats.t.sf(np.abs(t_stat), df)

t_crit = stats.t.ppf(1 - alpha/2, df)
margin = t_crit * se
ci_lower = x̄ - margin
ci_upper = x̄ + margin

print(f"t: {round(t_stat, 4)}") # test statistic
print(f"p: {round(p_value, 4)}") # p value
print(f"{(1-alpha)*100:.0f}% CI for mean: ({round(ci_lower, 4)}, {round(ci_upper, 4)})")

## Welch Two-Sample t-test

This is a hypothesis test of the following hypotheses:
$$\begin{gather}
\delta=\mu_2-\mu_1,\\
H_0:~\delta=\delta_0,~~H_1:~\delta\neq\delta_0
\end{gather}$$
The Welch two-sample t-test statistic is given by:
$$
t_\text{obs}=\frac{(\bar x_1 - \bar x_2)-\delta_0}{\sqrt{s^2_1/n_1~+~s^2_2/n_2}}
$$
The degrees of freedom for this test is given by:
$$\nu \approx \frac{\left( \frac{s_1^2}{n_1} + \frac{s_2^2}{n_2} \right)^2}
{\frac{\left( \frac{s_1^2}{n_1} \right)^2}{n_1 - 1} + \frac{\left( \frac{s_2^2}{n_2} \right)^2}{n_2 - 1}}$$
where $s^2$ is sample variances. (Variance is not assumed equal)</br>
$p>\alpha$ means we fail to reject $H_0$, and there is no significant difference.

In [ ]:
xA = np.array([7.53, 7.48, 8.08, 8.09, 10.15, 8.4, 10.88, 6.13, 7.9])
xB = np.array([9.21, 11.51, 12.79, 11.85, 9.97, 8.79, 9.69, 9.68, 9.19])

test = stats.ttest_ind(xB,xA,equal_var=False)
print(f"t: {test.statistic}   p: {test.pvalue}   df: {test.df}")

In [ ]:
x̄1 = 23/90
std1 = np.sqrt(23/90*(1-23/90))
n1 = 90

x̄2 = 35/80
std2 = np.sqrt(35/80*(1-35/80))
n2 = 80

x = 0 # expected difference, 0 for x̄1=x̄2
alpha = 0.05

# Standard error
se_diff = np.sqrt((std1**2 / n1) + (std2**2 / n2))

# t-statistic for H0: (μ1 - μ2) = x
t_statistic = ((x̄1 - x̄2) - x) / se_diff

# Degrees of freedom using Welch–Satterthwaite equation
df_num = (std1**2 / n1 + std2**2 / n2) ** 2
df_den = ((std1**2 / n1) ** 2) / (n1 - 1) + ((std2**2 / n2) ** 2) / (n2 - 1)
df = df_num / df_den

# Confidence Interval for the actual difference in means
diff = x̄1 - x̄2
CI_low = diff - stats.t.ppf(1 - alpha/2, df) * se_diff
CI_high = diff + stats.t.ppf(1 - alpha/2, df) * se_diff

# Two-tailed p-value
p_value = 2 * stats.t.sf(np.abs(t_statistic), df)

print(f"t: {t_statistic:.4f}   p: {p_value:.4f}   df: {df:.2f}")
print(f"Expected Difference in means: {diff:.4f}")
print(f"Confidence Interval: ({CI_low:.4f}, {CI_high:.4f})")

## Pooled Two-Sample t-test (Student's t-test)

As seen in Welch t-test but variances are combined for higher accuracy.</br>
$p>\alpha$ means we fail to reject $H_0$, and there is no significant difference.

In [ ]:
xA = np.array([7.53, 7.48, 8.08, 8.09, 10.15, 8.4, 10.88, 6.13, 7.9])
xB = np.array([9.21, 11.51, 12.79, 11.85, 9.97, 8.79, 9.69, 9.68, 9.19])

test = stats.ttest_ind(xB,xA)
print(f"t: {test.statistic}   p: {test.pvalue}   df: {test.df}")

In [ ]:
x̄1 = 23/90
std1 = np.sqrt(23/90*(1-23/90))
n1 = 90

x̄2 = 35/80
std2 = np.sqrt(35/80*(1-35/80))
n2 = 80

sp2 = (((n1 - 1) * std1**2) + ((n2 - 1) * std2**2)) / (n1 + n2 - 2) # pooled variance
se = np.sqrt(sp2 * (1/n1 + 1/n2)) # standard error
t_stat = (x̄2 - x̄1) / se
df = n1 + n2 - 2
p_val = 2 * stats.t.sf(np.abs(t_stat), df)

diff = x̄1 - x̄2
se_diff = np.sqrt((std1**2 / n1) + (std2**2 / n2))
CI_low = diff - stats.t.ppf(1-alpha/2, df) * se_diff
CI_high = diff + stats.t.ppf(1-alpha/2, df) * se_diff

print(f"t: {round(t_stat, 4)}   p: {round(p_val, 4)}   df: {df}")
print(f"Expected Difference in means: {diff:.4f}")
print(f"Confidence Interval: ({CI_low:.4f}, {CI_high:.4f})")

## Paired t-Test

This is used to determine whether two linked samples are statistically different. This may be measurements taken at two different times, locations, etc.</br>
As an example, drugs A and B are being tested. Each are tested on the same 10 people. A measurable effect is noted for each person.</br>
We can tell whether the drugs have statistically different effects like so:

In [ ]:
A = np.array([0.7, -1.6, -0.2, -1.2, -1.0, 3.4, 3.7, 0.8, 0.0, 2.0]) # Ordered values
B = np.array([1.9, 0.8, 1.1, 0.1, -0.1, 4.4, 5.5, 1.6, 4.6, 3.4])

test = stats.ttest_rel(B,A)
print(f"t: {test.statistic}   p: {test.pvalue}   df: {test.df}")

## ANOVA Testing

A statistical test used to analyze the difference between the means of more than two groups.</br>
It has the following two hypotheses:
- $H_0:\mu_1=\mu_2=\dots=\mu_n$
- $H_1:\text{not }H_0$

| Source         | df  | SS     | MS                                | F                     | p value    |
|----------------|-----|--------|-----------------------------------|-----------------------|------------|
| Between Groups | a-1 | SS$_B$ | MS$_B=\frac{\text{SS}_B}{a-1}$    | F=$\frac{MS_B}{MS_E}$ | P(F>F$_0$) |
| Within Groups  | n-a | SS$_Q$ | MS$_E=\frac{\text{SS}_Q}{a(n-1)}$ |                       |            |

**df** (degrees of freedom) is given for the test overall ('a': number of groups), as well as for within each group ('n': total observations). Using this, the total observations can be found as df$_\text{between}$ + df$_\text{within}$ + 1.

**SS** (sum of squares) represents the total difference in the sample. The SS between groups tells us the difference between groups, and the SS within groups represents random error. A larger between-group SS therefore implies significant difference between some groups.

**MS** (mean square) is used to calculate the F-statistic. It has the same trend as the SS.

**F-statistic** is the test statistic for the hypotheses of this test. A value >1 suggests a real difference between groups, and evidence against $H_0$.

**p-value** is the probability of seeing these results given that the true means are all equal. A value >$\alpha$ (usually 0.05) means at least one group differs significantly, and we reject $H_0$.

---

The test uses the following model for the data:
$$
Y_{ij}=\mu_i+\epsilon_{ij},~~\epsilon_{ij}\sim N(0,\sigma^2)
$$
This model makes the following two assumptions:
- Values within each group (and therefore residuals) are normally distributed
- Variance is the same in each group

If a significant difference is found, an Ad-Hoc test should be conducted to find which specific groups differ.

The effect of each group can be estimated by subtracting that group's mean from the mean of all groups.</br>
$MS_E$ is an approximation of the variance, thus the square root of this approximates the standard deviation.

In [ ]:
# Example Anova Test
values = [
    253.7, 241.2, 255.8, 249.3,         # Group 1 (4 obs)
    261.1, 244.2, 250.5, 264.9, 259.3,  # Group 2 (5 obs)
    257.9, 263.5, 258.6,                # Group 3 (3 obs)
    244.1, 244.9, 243.9, 247.1          # Group 4 (4 obs)
]

# repeat the group labels to match counts
locs = np.repeat([1, 2, 3, 4], [4, 5, 3, 4])

data = pd.DataFrame({
    'values': values,
    'group' : pd.Categorical(locs)
})

fit = smf.ols('values ~ group', data=data).fit()
anova = sm.stats.anova_lm(fit)
anova

In [ ]:
# ANOVA with summary statistics

# Input: summary statistics for each group
# Each group should be a dictionary with 'n' (sample size), 'mean', and 'variance'
groups = [
    {'n': 325, 'mean': 202/325, 'variance': 202/325*(1-202/325)},
    {'n': 285, 'mean': 117/285, 'variance': 117/285*(1-117/285)},
    {'n': 706, 'mean': 178/706, 'variance': 178/706*(1-178/706)},
    {'n': 885, 'mean': 212/885, 'variance': 212/885*(1-212/885)}
]

# Total number of observations and grand mean
N = sum(g['n'] for g in groups)
grand_mean = sum(g['n'] * g['mean'] for g in groups) / N

# Between-group sum of squares (SSB)
SSB = sum(g['n'] * (g['mean'] - grand_mean) ** 2 for g in groups)
df_between = len(groups) - 1 # df of group
MSB = SSB / df_between

# Within-group sum of squares (SSW)
SSW = sum((g['n'] - 1) * g['variance'] for g in groups)
df_within = N - len(groups) # df of residuals
MSW = SSW / df_within

# F-statistic
F = MSB / MSW
p_value = 1 - stats.f.cdf(F, df_between, df_within)

# Output
print(f"F-statistic: {F:.4f}")
print(f"Degrees of freedom: ({df_between}, {df_within})")
print(f"p-value: {p_value:.4f}")


#### Ad-Hoc
This test can determine which of a set of ≥3 samples have means which are significantly different from each-other. This uses the CI width (identical for each mean) to determine which CIs overlap.

The following formulas calculate the LSD (CI width). If any means are more than this far apart, they are considered significantly different.
For this test, each mean must be compared to all others. The number of this copmparison is equal to $\sum_{k=1}^{a-1}a$.
$$\begin{gather}
\alpha_\text{Bonferroni}=\alpha~/~\tiny\#\normalsize\text{ of comparisons} \\
LSD=\text{ppf}\left(\frac{1-\alpha_\text{Bonferroni}}2,\text{df}_\text{resid}\right)\cdot\sqrt{\frac{2\cdot \text{sq}_\text{resid}}{n/a}}
\end{gather}$$

In [ ]:
# LSD from summary statistics
num_groups = 10         # Number of ANOVA groups
samples_per_group = 6   # Average number of measurements per group
mse = 10                # Mean Square Error from ANOVA (residual)
df_error = 50           # Degrees of Freedom for residual
alpha = 0.05            # Significance level

t_crit = stats.t.ppf(1 - alpha/2, df_error)
lsd = t_crit * np.sqrt(mse * (2 / samples_per_group))

print(f"LSD: {round(lsd, 4)}")
print("Any pairwise mean difference greater than this is statistically significant.")


In [ ]:
# Full Ad-Hoc (Post-Hoc) Test
group_means = {
    'A': -1.28666,
    'B': -0.541666,
    'C': -1.13666,
    'D': -0.681666
}
group_sizes = {
    'A': 6,
    'B': 6,
    'C': 6,
    'D': 6
}

mse = 0.16         # Mean Square Error from ANOVA (residual)
df_error = 20     # Degrees of freedom of the residual
alpha = 0.05

t_crit = stats.t.ppf(1 - alpha/2, df_error)

def compute_lsd(mse, n1, n2, t_crit):
    return t_crit * np.sqrt(mse * (1/n1 + 1/n2))

# Pairwise comparisons
groups = list(group_means.keys())
results = []

for i in range(len(groups)):
    for j in range(i+1, len(groups)):
        g1, g2 = groups[i], groups[j]
        mean_diff = group_means[g2] - group_means[g1]
        lsd = compute_lsd(mse, group_sizes[g1], group_sizes[g2], t_crit)
        significant = abs(mean_diff) > lsd
        ci_lower = mean_diff - lsd
        ci_upper = mean_diff + lsd
        results.append({
            'Group 1': g1,
            'Group 2': g2,
            'Mean Diff': round(mean_diff, 4),
            'LSD': round(lsd, 4),
            f"{int((1-alpha)*100)}% CI": f"[{round(ci_lower, 4)}, {round(ci_upper, 4)}]",
            'Significant': 'Yes' if significant else 'No'
        })

df_results = pd.DataFrame(results)
print(df_results.to_string(index=False))


A table can be created with all means in increasing order. Letters can be given to each mean. If means share letters, they are not significantly different. (Figure out a method for that lmao)

Example:
| Group | Mean | |
|---|---|---|
| 5 | 44.3 | a |
| 1 | 47.9 | ab |
| 4 | 50.9 | bc |
| 3 | 52.8 | cd |
| 2 | 56.1 | d |

This shows which groups are significantly different from which others.

## ANOVA (Two-way analysis of variance)

In [ ]:
# Data
y = np.array([2.8, 3.6, 3.4, 2.3,
              5.5, 6.3, 6.1, 5.7,
              5.8, 8.3, 6.9, 6.1])
treatm = pd.Categorical([1, 1, 1, 1,
                         2, 2, 2, 2,
                         3, 3, 3, 3])
block = pd.Categorical([1, 2, 3, 4,
                        1, 2, 3, 4,
                        1, 2, 3, 4])

# Create DataFrame
D = pd.DataFrame({'y': y, 'treatm': treatm, 'block': block})

# Model: Two-way ANOVA without interaction
model = smf.ols('y ~ C(treatm) + C(block)', data=D).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print("Two-way ANOVA without interaction:")
print(anova_table)

# Optional: With interaction
model_interact = smf.ols('y ~ C(treatm) * C(block)', data=D).fit()
anova_table_interact = sm.stats.anova_lm(model_interact, typ=2)
print("\nTwo-way ANOVA with interaction:")
print(anova_table_interact)

## Chi-Square Test

Under $H_0$, two variables are independent. $H_1$ states that they are not.

$p<\alpha$ means we **reject** $H_0$

In [ ]:
# Example Chi-Square test
alpha = 0.05
observed = pd.DataFrame({
    "p": [11, 27, 15],
    "m": [14, 40, 30],
    "g": [5, 23, 35]
}, index=["ba", "a", "aa"])

# Perform the Chi-square test for independence
chi2, p, dof, expected = stats.chi2_contingency(observed)
if alpha:
    critical_val = stats.chi2.ppf(1 - alpha, dof)

# Prepare results
print(f"chi-value: {chi2:.4f}, p: {p:.8f}, dof: {dof}")
if alpha:
    print(f"Critical Value at a={alpha}: {critical_val:.4f}")
    print("Variables are independent (Reject H1)" if p>alpha else "Variables are not independent (Reject H0)")
print(f"Expected values if independent:")
pd.DataFrame(expected, index=observed.index, columns=observed.columns)

# Model Validation


## Test Powers
Example: A burglary alarm system has 80% power: 80% chance of detecting a burglary. (20% chance of *type II error*)</br>
Sigma is the difference being looked for divided by the effect size.

In [ ]:
# Fixing the parameter name for TTestIndPower (should be `nobs1` instead of `nobs`)
from statsmodels.stats.power import TTestPower, TTestIndPower

def solve_ttest_power(test_type='1-sample', effect_size=None, nobs=None, alpha=0.05, power=None, alternative='two-sided', ratio=1.0):
    """
    Solves for one missing parameter in a 1-sample or 2-sample t-test power analysis.

    Parameters:
    - test_type: '1-sample' or '2-sample'
    - effect_size: Cohen's d (standardized effect size)
    - nobs: Sample size (per group for 2-sample)
    - alpha: Significance level (e.g., 0.05)
    - power: Desired power (e.g., 0.8)
    - alternative: 'two-sided', 'larger', or 'smaller'
    - ratio: Sample size ratio n2/n1 for 2-sample test (default = 1.0)

    Returns:
    - Computed missing value
    """
    if test_type == '1-sample':
        analysis = TTestPower()
        result = analysis.solve_power(effect_size=effect_size, nobs=nobs, alpha=alpha, power=power, alternative=alternative)
    elif test_type == '2-sample':
        analysis = TTestIndPower()
        result = analysis.solve_power(effect_size=effect_size, nobs1=nobs, alpha=alpha, power=power, alternative=alternative, ratio=ratio)
    else:
        raise ValueError("test_type must be '1-sample' or '2-sample'")

    return result

# Example usage:
one_sample_result = solve_ttest_power(test_type='1-sample', effect_size=0.5, alpha=0.05, power=0.8)
two_sample_result = solve_ttest_power(test_type='2-sample', effect_size=0.5, alpha=0.05, power=0.8)

print(solve_ttest_power(test_type='1-sample', effect_size=0.3/1.5, alpha=0.05, power=0.8, nobs=None)) # finds effect size